# University of Wisconsin-Madison - School of Business

Neda Jabbari, Erica Ho, Juan Julian Herranz
08/08/2019



Data reterived from:
Browse through the information page for the University of Wisconsin Madison 
https://www.kaggle.com/Madgrades/uw-madison-courses


Idenfify the courses related to the school of business.
Business courses code ref: https://bus.wisc.edu/current-student-resources/bba/study-abroad/incoming-exchange-students/courses 

## H1: Course instructors affect the ratio of A grades.
## H0: Course instructors have no effect on the ratio of A grades.

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as s
from scipy.stats import ttest_ind
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
conn = psycopg2.connect(host='localhost',
                        database='uw-madison-courses')
cur = conn.cursor()

To investigate a relationship between course instructors and grades in the school of business, obtain data from: course_offerings, grade_distribution, subject_memberships, sections, teachings

In [ ]:
bus_courses = ['231', '232', '233', '235', '236', '237', '238', '239', '241', '242']

Querry the database to retrieve information on the desired courses.

In [ ]:
cur.execute("""SELECT DISTINCT ON (s.course_offering_uuid, s.number)
                      co.uuid
                      ,co.name
                      ,gd.section_number
                      , gd.a_count
                      , gd.ab_count
                      , gd.b_count
                      , gd.bc_count
                      ,gd.c_count
                      , gd.d_count
                      , gd.f_count
                      ,s.uuid
                      ,t.instructor_id
               FROM course_offerings co
               INNER JOIN grade_distributions gd on co.uuid=gd.course_offering_uuid
               INNER JOIN subject_memberships sm on co.uuid=sm.course_offering_uuid
               INNER JOIN sections s ON co.uuid=s.course_offering_uuid
               INNER JOIN teachings t ON s.uuid=t.section_uuid
               WHERE sm.subject_code in ('231', '232', '233', '235', '236', '237', '238', '239', '241', '242')
               ;""")

business_courses = pd.DataFrame(cur.fetchall())
business_courses.columns = ['uuid', 'name', 'section_number', 'a_count',
                            'ab_count', 'b_count', 'bc_count', 'c_count', 'd_count', 'f_count', 'section_uuid', 'instructor_id']
business_courses.head(5)

In [ ]:
sum(business_courses.duplicated())

In [ ]:
cur.execute("""SELECT DISTINCT co1.uuid,
                      count(distinct t.instructor_id) as teachers
               FROM course_offerings co1
               INNER JOIN subject_memberships sm on co1.uuid=sm.course_offering_uuid
               INNER JOIN sections s ON co1.uuid=s.course_offering_uuid
               INNER JOIN teachings t ON s.uuid=t.section_uuid
               WHERE sm.subject_code in ('231', '232', '233', '235', '236', '237', '238', '239', '241', '242')
               GROUP BY co1.uuid, sm.subject_code
               ORDER BY teachers DESC
               ;""")

b_c_tchr_df = pd.DataFrame(cur.fetchall())
b_c_tchr_df.columns = [i[0] for i in cur.description]
b_c_tchr_df.head()

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
sns.distplot(business_courses['instructor_id'].value_counts());

In [ ]:
business_courses.instructor_id.nunique()

### Data engineering to include the total number of students in order to generate the As ratio

Given that the grade columns are all strings, we need to convert them into integers in order to manipulate them

In [ ]:
grade_cols = ['a_count', 'ab_count', 'b_count', 'bc_count', 'c_count', 'd_count', 'f_count']

for grade_col in grade_cols:
    business_courses[grade_col] = business_courses[grade_col].apply(lambda x: int(x))

In [ ]:
business_courses['n_students'] = (business_courses['a_count'] + business_courses['ab_count'] 
                                         + business_courses['b_count'] + business_courses['bc_count']
                                         + business_courses['c_count'] + business_courses['d_count']
                                         + business_courses['f_count'])

In [ ]:
business_courses.head(2)
business_courses.shape

In [ ]:
business_courses = business_courses[business_courses['n_students']!= 0]

In [ ]:
business_courses.shape

In [ ]:
business_courses['ratio_of_a_grades'] = business_courses['a_count'] / business_courses['n_students']

### Performing ANOVA test

In [ ]:
formula = 'ratio_of_a_grades ~ instructor_id'
lm = ols(formula, business_courses).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

ANOVA test suggests that Course instructors affect the ratio of A grades.

## H1: The average grade among students who take online courses is lower than the rest of the students.
## H0: The average grade among students who take online courses is not different from other students.



In [ ]:
query = """
SELECT DISTINCT ON (sections.course_offering_uuid, sections.number)
  sections.uuid
, sections.course_offering_uuid
, sections.section_type
, sections.number AS section_number
, sections.room_uuid
, sections.schedule_uuid
-- , course_offering_uuid
-- , section_number
, n_count
, c_count
, f_count
, cr_count
, p_count
, nw_count
, other_count
, bc_count
, u_count
, d_count
, ab_count
, a_count
, s_count
, i_count
, b_count
, nr_count
-- , uuid
, facility_code
, room_code
,course_offerings.name
-- ,subject_code
FROM sections 
    INNER JOIN grade_distributions
        ON sections.course_offering_uuid = grade_distributions.course_offering_uuid
            AND sections.number = grade_distributions.section_number
    INNER JOIN course_offerings 
        ON sections.course_offering_uuid = course_offerings.uuid
    INNER JOIN subject_memberships
        ON sections.course_offering_uuid = subject_memberships.course_offering_uuid
    LEFT JOIN rooms ON sections.room_uuid = rooms.uuid
    WHERE subject_memberships.subject_code in ('231', '232', '233', '235', '236', '237', '238', '239', '241', '242')


;                      
"""

cur.execute(query)
df= pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head(5)

In [ ]:
sum(df.duplicated())   #no duplicated rows

In [ ]:
df = df.drop(
    ['course_offering_uuid','section_number','schedule_uuid','n_count','cr_count',
     'p_count','nw_count','other_count','u_count','s_count','i_count','nr_count'], axis=1)

In [ ]:
for i in list(df.columns[3:10]):df[i]=df[i].astype('int64')   #convert str grade counts to int

In [ ]:
count_cols = sorted(i for i in df.columns if i.endswith('_count'))

grade_points = [4.0, 3.5, 3.0, 2.5, 2.0, 1.0, 0.0]

count_col_values = dict(zip(count_cols, grade_points))
count_col_values

In [ ]:
grades_df = df.loc[:, count_col_values.keys()]

In [ ]:
grade_values = np.array(list(count_col_values.values()))

In [ ]:
grade_counts = grades_df.values
grade_counts[1]
sum(grade_counts[1])

In [ ]:
grade_points = grade_counts @ grade_values

In [ ]:
df['avg_grade_points'] = [grade_points[i]/sum(grade_counts[i]) 
                                                for i in range(len(grade_points))]

In [ ]:
df = df.dropna(subset = ['avg_grade_points'])

In [ ]:
df.loc[df['facility_code']=='ONLINE']['room_uuid'].unique()    #the room unique id for online courses is 86d1debf-3f3a-39b7-bc0d-976a4fa33862

In [ ]:
#to get the list of online courses 
online_df = df.loc[df['room_uuid']=='86d1debf-3f3a-39b7-bc0d-976a4fa33862']
online_df.head()
online_df.shape

In [ ]:
#to get the list of courses that are not online
non_online_df = df.loc[df['room_uuid']!='86d1debf-3f3a-39b7-bc0d-976a4fa33862']
non_online_df.head()
non_online_df.shape

In [ ]:
sns.set(rc={'figure.figsize':(15,10)})
plt.ylabel('distribution')
plt.title('Average Grade Ditribution')
sns.distplot(non_online_df['avg_grade_points']);  # Blue distribution
sns.distplot(online_df['avg_grade_points']);  # Orange distribution

In [ ]:
print(s.normaltest(non_online_df['avg_grade_points']))  #non_online avg grade points are not normally distributed
print(s.normaltest(online_df['avg_grade_points']))     #online avg grade points are not normally distributed

In [ ]:
non_online_sample_means= [non_online_df['avg_grade_points'].sample(frac = 1, replace = True).mean() 
                          for i in range(1000)]

In [ ]:
online_sample_means= [online_df['avg_grade_points'].sample(frac = 1, replace = True).mean() 
                          for i in range(1000)]

In [ ]:
plt.xlabel('mean of sampled average grades')
plt.ylabel('distribution')
plt.title('Average Grade Sample Means Ditribution')
sns.distplot(non_online_sample_means);
sns.distplot(online_sample_means);

In [ ]:
ttest_ind(non_online_sample_means, online_sample_means)  

With alpha of 0.05 and pvalue/2 < alpha, the average grade among students who take online courses is lower than the rest of the students.

## H1: There is a difference between lecture classes of different number of students in terms of average grades.
## H0: There is no difference between lecture classes of different number of students in terms of average grades.

In [ ]:
non_online_lecture_df= non_online_df[non_online_df["section_type"]=='LEC']

In [ ]:
count_cols = sorted(i for i in df.columns if i.endswith('_count'))

In [ ]:
non_online_lecture_df['class_size'] = non_online_lecture_df.loc[:, count_cols].sum(axis=1)

In [ ]:
sns.distplot(non_online_lecture_df['class_size']);  # Blue distribution

In [ ]:
mean_class_size = non_online_lecture_df['class_size'].mean() 

In [ ]:
below_mean_non_online_lecture_df = non_online_lecture_df.loc[non_online_lecture_df['class_size'] > 
                                                             mean_class_size]
above_mean_non_online_lecture_df = non_online_lecture_df.loc[non_online_lecture_df['class_size'] < 
                                                             mean_class_size]

In [ ]:
sns.distplot(below_mean_non_online_lecture_df['avg_grade_points']);  # Blue distribution
sns.distplot(above_mean_non_online_lecture_df['avg_grade_points']);

In [ ]:
#check if the data is normally distributed
print(s.normaltest(below_mean_non_online_lecture_df['avg_grade_points'])) 

print(s.normaltest(above_mean_non_online_lecture_df['avg_grade_points']))

In [ ]:
below_mean_non_online_lecture_sample_means= [below_mean_non_online_lecture_df['avg_grade_points'].sample(frac = 1, replace = True).mean() 
                          for i in range(1000)]

In [ ]:
above_mean_non_online_lecture_sample_means= [above_mean_non_online_lecture_df['avg_grade_points'].sample(frac = 1, replace = True).mean() 
                          for i in range(1000)]

In [ ]:
sns.distplot(below_mean_non_online_lecture_sample_means);
sns.distplot(above_mean_non_online_lecture_sample_means);

In [ ]:
ttest_ind(below_mean_non_online_lecture_sample_means, above_mean_non_online_lecture_sample_means)  

Based on the P-value and the alpha of 0.05, there is a difference between lecture classes of different number of students in terms of average grades.

## H1: There is a difference between grades offered in Fall term and those in Spring term.
## H0: There is no difference between grades offered in Fall term and those in Spring term.

In [ ]:
cur.execute ( """SELECT
DISTINCT ON (sections.course_offering_uuid, sections.number)
  sections.uuid
, sections.course_offering_uuid
, sections.section_type
, sections.number AS section_number
, sections.schedule_uuid
, facility_code
, term_code
, course_offerings.name
, a_count
, ab_count
, b_count
, bc_count
, c_count
, d_count
, f_count
-- ,subject_code
-- , course_offering_uuid
-- , section_number
-- , uuid
FROM sections
   INNER JOIN grade_distributions
       ON sections.course_offering_uuid = grade_distributions.course_offering_uuid
           AND sections.number = grade_distributions.section_number
   INNER JOIN course_offerings
       ON sections.course_offering_uuid = course_offerings.uuid
   INNER JOIN subject_memberships
       ON sections.course_offering_uuid = subject_memberships.course_offering_uuid
   LEFT JOIN rooms ON sections.room_uuid = rooms.uuid
WHERE subject_memberships.subject_code in ('232', '233', '231', '237', '238', '242', '236', '239', '235','241')
;""")
course_grades_sub_df= pd.DataFrame(cur.fetchall())
course_grades_sub_df.columns = [i[0] for i in cur.description]

In [ ]:
course_grades_sub_df['c_count'] = course_grades_sub_df['c_count'].astype(int)
course_grades_sub_df['bc_count'] = course_grades_sub_df['bc_count'].astype(int)
course_grades_sub_df['d_count'] = course_grades_sub_df['d_count'].astype(int)
course_grades_sub_df['ab_count'] = course_grades_sub_df['ab_count'].astype(int)
course_grades_sub_df['a_count'] = course_grades_sub_df['a_count'].astype(int)
course_grades_sub_df['b_count'] = course_grades_sub_df['b_count'].astype(int)
course_grades_sub_df['f_count'] = course_grades_sub_df['f_count'].astype(int)

In [ ]:
course_grades_sub_df['total_num_student'] = (course_grades_sub_df['a_count'] 
                                             + course_grades_sub_df['ab_count'] 
                                             + course_grades_sub_df['b_count']
                                             + course_grades_sub_df['bc_count'] 
                                             + course_grades_sub_df['c_count'] 
                                             + course_grades_sub_df['d_count'] 
                                             + course_grades_sub_df['f_count'])

In [ ]:
course_grades_sub_df['avg_grade'] = (course_grades_sub_df['a_count']*4 + course_grades_sub_df['ab_count']*3.5 
+ course_grades_sub_df['b_count']*3 + course_grades_sub_df['bc_count']*2.5 + course_grades_sub_df['c_count']*2
+ course_grades_sub_df['d_count']*1)/course_grades_sub_df['total_num_student']

As the analysis is solely based on courses that are graded, the dataframe only includes classes that have grades 

In [ ]:
course_grades_sub_df = course_grades_sub_df.loc[~(course_grades_sub_df['total_num_student']==0),:]

In [ ]:
course_grades_sub_df['term_code'] = course_grades_sub_df['term_code'].astype(str)

In [ ]:
def term(term_code):
    if term_code[-1] == '2':
        return 'fall'
    elif term_code[-1] == '4':
        return 'spring'
    else:
        return 'other'           
        
course_grades_sub_df['term'] = course_grades_sub_df['term_code'].apply(term)

In [ ]:
other_df = course_grades_sub_df.loc[course_grades_sub_df['term'] == 'other']
len(other_df)

In [ ]:
fall_df = course_grades_sub_df.loc[course_grades_sub_df['term'] == 'fall']
len(fall_df)

In [ ]:
spring_df = course_grades_sub_df.loc[course_grades_sub_df['term'] == 'spring']
len(spring_df)

Fall term dataframe

In [ ]:
fall_sample_mean = sum(fall_df['avg_grade'])/len(fall_df)

In [ ]:
sns.distplot(fall_df['avg_grade'])

Spring term dataframe

In [ ]:
spring_sample_mean = sum(spring_df['avg_grade'])/len(spring_df)

In [ ]:
sns.distplot(spring_df['avg_grade'])

Bootstrapping for fall term and spring term

In [ ]:
bootstrap_samples_fall = []
bootstrap_sample_means_fall = np.zeros(3168)
for i in range(3168):
    bootstrap_sample = np.random.choice(fall_df['avg_grade'], size=3168)
    bootstrap_samples_fall.append(bootstrap_sample)
    bootstrap_sample_means_fall[i] = bootstrap_sample.mean()

In [ ]:
bootstrap_samples_spring = []
bootstrap_sample_means_spring = np.zeros(2540)
for i in range(2540):
    bootstrap_sample = np.random.choice(spring_df['avg_grade'], size=2540)
    bootstrap_samples_spring.append(bootstrap_sample)
    bootstrap_sample_means_spring[i] = bootstrap_sample.mean()

In [ ]:
fig, ax = plt.subplots()
sns.distplot(bootstrap_sample_means_fall)
sns.distplot(bootstrap_sample_means_spring)
plt.title('Grades in Fall and Spring Terms')
plt.xlabel('Grades')
plt.ylabel('Number of students');

In [ ]:
print(s.ttest_ind(spring_df['avg_grade'], fall_df['avg_grade']))

Based on the P-value and the alpha of 0.05, there is no difference between grades offered in Fall term and those in Spring term